In [1]:
!git clone https://github.com/Sugar-Care/ML_Stuff.git

Cloning into 'ML_Stuff'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 58 (delta 16), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 2.84 MiB | 5.41 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/content/ML_Stuff/Sulthan_Stuff/dataset_cleaned.csv')

In [4]:
X = df[['Age', 'Blood Glucose Levels', 'Blood Pressure', 'Weight Gain During Pregnancy',
        'Waist Circumference', 'BMI', 'Insulin Levels', 'Cholesterol Levels',
        'Digestive Enzyme Levels', 'Pulmonary Function']]
y = df['Target']

# Split data into training and temporary sets (80% train, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42) #random_state for reproducibility

# Split the temporary set into validation and testing sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Now you have:
print(len(X_train), len(y_train))
print(len(X_val), len(y_val))
print(len(X_test), len(y_test))

16354 16354
2044 2044
2045 2045


In [5]:
sulthan =  tf.keras.Sequential([
    tf.keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Conv1D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(df['Target'].unique()), activation='softmax')
])
sulthan.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mae', 'mse'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
sulthan.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 10, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 8, 32)               │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 4, 32)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 2, 64)               │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 13)                  │           1,677 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,333 (63.80 KB)

 Trainable params: 16,333 (63.80 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Training the model
history = sulthan.fit(
    X_train,
    y_train,
    epochs=200,
    batch_size=128,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2162 - loss: 3.8067 - mae: 5.9215 - mse: 49.0835 - val_accuracy: 0.5245 - val_loss: 1.2800 - val_mae: 5.9226 - val_mse: 48.9576
Epoch 2/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5235 - loss: 1.2611 - mae: 5.9623 - mse: 49.5222 - val_accuracy: 0.6267 - val_loss: 0.9374 - val_mae: 5.9226 - val_mse: 48.9674
Epoch 3/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6093 - loss: 0.9982 - mae: 5.9273 - mse: 49.1298 - val_accuracy: 0.6659 - val_loss: 0.8378 - val_mae: 5.9226 - val_mse: 48.9717
Epoch 4/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6375 - loss: 0.9166 - mae: 5.9253 - mse: 49.2097 - val_accuracy: 0.6551 - val_loss: 0.8454 - val_mae: 5.9226 - val_mse: 48.9741
Epoch 5/200
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6530 - loss: 0.8661 - mae: 5.9142 - mse: 49.0922 - val_accuracy: 0.7045 - val_loss: 0.7454 - val_mae: 5.9226 - val_mse: 48.9755
Epoch 6/200
128/128 ━━━━━

In [8]:
loss, accuracy, mae, mse = sulthan.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test MSE: {mse:.4f}")

y_pred = sulthan.predict(X_test)
y_pred_classes = tf.argmax(y_pred, axis=1)
y_pred_probs = np.max(y_pred, axis=1)

print("\nExample Predictions:")
for i in range(20):
    print(f"Sample {i+1}: Predicted Class - {y_pred_classes[i]}, Actual Class - {y_test.iloc[i]}, Probability - {y_pred_probs[i]:.4f}")

Test Loss: 0.3960
Test Accuracy: 0.8377
Test MAE: 5.9175
Test MSE: 48.5709
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Example Predictions:
Sample 1: Predicted Class - 7, Actual Class - 7, Probability - 0.7061
Sample 2: Predicted Class - 3, Actual Class - 3, Probability - 0.9090
Sample 3: Predicted Class - 4, Actual Class - 4, Probability - 1.0000
Sample 4: Predicted Class - 10, Actual Class - 10, Probability - 0.7269
Sample 5: Predicted Class - 2, Actual Class - 2, Probability - 1.0000
Sample 6: Predicted Class - 7, Actual Class - 9, Probability - 0.6924
Sample 7: Predicted Class - 12, Actual Class - 12, Probability - 0.9998
Sample 8: Predicted Class - 8, Actual Class - 8, Probability - 0.9398
Sample 9: Predicted Class - 11, Actual Class - 11, Probability - 1.0000
Sample 10: Predicted Class - 1, Actual Class - 1, Probability - 0.9955
Sample 11: Predicted Class - 11, Actual Class - 11, Probability - 0.9989
Sample 12: Predicted Class - 4, Actual Class - 4, Probability - 1.0000
Sample 13: Pr

In [9]:
sulthan.save("sulthan_cnn_model.keras")